In [1]:
import sys
import csv
import bz2

import numpy as np
import cv2
import pickle
from PIL import Image

from colormath.color_diff_matrix import delta_e_cie2000
from colormath.color_objects import LabColor

from skimage import img_as_float, img_as_ubyte
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.filters import sobel
from skimage.segmentation import mark_boundaries

In [2]:
infile = open('/media/allen/mass/lab-matrix.pk','rb')
lab_matrix = pickle.load(infile, encoding='latin1')
infile.close()

infile = open('/media/allen/mass/lab-colors.pk','rb')
lab_colors = pickle.load(infile, encoding='latin1')
infile.close()

In [5]:
len(lab_colors)

197525

In [7]:
lab_colors = np.array(lab_colors)

In [51]:
color_centers = []
for color in np.unique(lab_colors):
    mask = lab_colors == color
    step = mask.sum() / (mask.sum()*0.01)
    samples = lab_matrix[mask][::100]
    color_centers.append(samples)

In [60]:
np.unique(lab_colors)[0]

'beige'

In [62]:
np.random.uniform(0,)

<function RandomState.uniform>

In [23]:
cie_lab = np.array([color_center])
cv_lab = np.zeros_like(cie_lab)
cv_lab[...,0] = cie_lab[...,0] * 255 / 100
cv_lab[...,1] = cie_lab[...,1] - 128
cv_lab[...,2] = cie_lab[...,2] - 128
cv_lab = cv_lab.astype(np.uint8)

In [65]:
delta = delta_e_cie2000(cie_lab[0][0], lab_matrix)

In [72]:
set(lab_colors[delta < 3])

{'beige', 'yellow'}

In [25]:
cv_bgr = cv2.cvtColor(cv_lab, cv2.COLOR_LAB2BGR)

In [30]:
for b, g, r in np.squeeze(cv_bgr):
    print(b,g,r)

0 24 37
9 8 11
170 65 41
4 61 107
219 231 31
0 134 174
184 187 189
37 164 66
0 55 57
23 1 82
0 105 219
144 0 224
141 0 123
21 13 186
133 157 15
58 66 67
1 202 211


In [35]:
mask = lab_colors == 'blue'

In [38]:
from sklearn.cluster import DBSCAN


In [45]:
lab_matrix[mask,:]

array([[  8.30566263,  39.85143396, -54.28106616],
       [  8.47604225,  40.13078748, -54.66156958],
       [  8.64612588,  40.40965569, -55.04141196],
       ...,
       [ 37.39302572,  81.0878837 , -98.673802  ],
       [ 37.50999473,  81.30873314, -99.0397929 ],
       [ 71.97153117,  -4.94462472, -42.78225755]])

In [48]:
lab_matrix[mask,:][::100]

array([[   8.30566263,   39.85143396,  -54.28106616],
       [  24.22342107,   65.95010525,  -89.82969169],
       [   2.85172837,   20.03926306,  -34.84045863],
       ...,
       [  31.42609893,   77.58016955, -105.38558547],
       [  35.28964276,   78.24525784,  -96.00054788],
       [  35.69403816,   78.44510764,  -95.3168025 ]])